## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from keras import regularizers


Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [3]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder(categories='auto')
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

In [4]:
classifier=Sequential()

#卷積組合
#'自己設計參數' eg:32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(Convolution2D(32, (3, 3), input_shape=(32, 32, 3), activation='relu'))
classifier.add(BatchNormalization())

'''自己決定MaxPooling2D放在哪裡'''
classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(32, (3, 3), activation='relu')) #'自己設計參數'
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2,2)))

#flatten
classifier.add(Flatten())

#FC
#'自己設計FC層參數'
classifier.add(Dense(output_dim=100, activation='relu', kernel_regularizer=regularizers.l2(l=0.001))) #output_dim=100,activation=relu
classifier.add(BatchNormalization())
classifier.add(Dropout(p=0.5)) #Dropout

classifier.add(Dense(output_dim=100, activation='relu', kernel_regularizer=regularizers.l2(l=0.001)))
classifier.add(BatchNormalization())
classifier.add(Dropout(p=0.3))

#輸出
classifier.add(Dense(output_dim=10, activation='softmax')) #'輸出函數應該用什麼？'


#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
#classifier.fit(x_train,y_train,batch_size=100,epochs=100)













Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", kernel_regularizer=<keras.reg..., units=100)`
D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.5)`
D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", kernel_regularizer=<keras.reg..., units=100)`
D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.3)`


D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`


In [5]:
from keras.preprocessing.image import ImageDataGenerator #Augmentation
img_gen = ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True, rotation_range=10,
                            width_shift_range=0.1, height_shift_range=0.1, shear_range=0.1,
                            zoom_range=0.1, horizontal_flip=True, vertical_flip=False, dtype=np.float32)
img_gen.fit(x_train)

In [6]:
from keras.callbacks import EarlyStopping
earlystop = EarlyStopping(monitor='test_loss', patience=8, verbose=1) #earlystop

#開始訓練
classifier.fit_generator(img_gen.flow(x_train, y_train, batch_size=100), steps_per_epoch=500,
                        epochs=100, validation_data = (x_test, y_test), callbacks = [earlystop])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/100
500/500 [==============================] - 24s 47ms/step - loss: 2.1226 - acc: 0.3461 - val_loss: 1.5928 - val_acc: 0.5115
Epoch 2/100
  9/500 [..............................] - ETA: 7s - loss: 1.8386 - acc: 0.4289

D:\Program Files\Anaconda3\lib\site-packages\keras\callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `test_loss` which is not available. Available metrics are: val_loss,val_acc,loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


500/500 [==============================] - 21s 43ms/step - loss: 1.6753 - acc: 0.4749 - val_loss: 1.3894 - val_acc: 0.5663
Epoch 3/100
500/500 [==============================] - 22s 43ms/step - loss: 1.4847 - acc: 0.5371 - val_loss: 1.3085 - val_acc: 0.5912
Epoch 4/100
500/500 [==============================] - 21s 43ms/step - loss: 1.3782 - acc: 0.5690 - val_loss: 1.1726 - val_acc: 0.6443
Epoch 5/100
500/500 [==============================] - 21s 43ms/step - loss: 1.3040 - acc: 0.5913 - val_loss: 1.1290 - val_acc: 0.6564
Epoch 6/100
500/500 [==============================] - 21s 43ms/step - loss: 1.2549 - acc: 0.6044 - val_loss: 1.0953 - val_acc: 0.6607
Epoch 7/100
500/500 [==============================] - 22s 44ms/step - loss: 1.2226 - acc: 0.6131 - val_loss: 1.1011 - val_acc: 0.6571
Epoch 8/100
500/500 [==============================] - 21s 43ms/step - loss: 1.1989 - acc: 0.6252 - val_loss: 1.1845 - val_acc: 0.6375
Epoch 9/100
500/500 [==============================] - 21s 42ms/ste

500/500 [==============================] - 21s 42ms/step - loss: 0.9857 - acc: 0.7032 - val_loss: 0.9242 - val_acc: 0.7265
Epoch 63/100
500/500 [==============================] - 21s 42ms/step - loss: 0.9842 - acc: 0.7049 - val_loss: 0.9689 - val_acc: 0.7119
Epoch 64/100
500/500 [==============================] - 21s 43ms/step - loss: 0.9772 - acc: 0.7061 - val_loss: 0.9375 - val_acc: 0.7186
Epoch 65/100
500/500 [==============================] - 21s 42ms/step - loss: 0.9802 - acc: 0.7059 - val_loss: 0.8975 - val_acc: 0.7334
Epoch 66/100
500/500 [==============================] - 21s 43ms/step - loss: 0.9832 - acc: 0.7041 - val_loss: 0.8992 - val_acc: 0.7339
Epoch 67/100
500/500 [==============================] - 22s 44ms/step - loss: 0.9789 - acc: 0.7069 - val_loss: 1.0266 - val_acc: 0.6939
Epoch 68/100
500/500 [==============================] - 21s 43ms/step - loss: 0.9824 - acc: 0.7031 - val_loss: 0.9956 - val_acc: 0.7022
Epoch 69/100
500/500 [==============================] - 21s 4

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [7]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[9.1704670e-03, 7.3058021e-05, 4.5072636e-01, 6.7761838e-02,
        4.1899461e-01, 7.3583652e-03, 3.9801862e-02, 2.7033640e-03,
        3.3141770e-03, 9.5944168e-05]], dtype=float32)